Package imports, etc.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import collections
from functools import reduce

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import time
import re

import pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

pd.set_option('display.max_columns', 30)

open selenium, go to quiz central

In [2]:
service = Service('chromedriver')
driver = webdriver.Chrome()
driver.get('https://quizcentral.net/qc/Online_Quiz_League_USA_-_Results_-_Season_1_-_Week_1')

some functions i wrote to simplify the scraping process

In [22]:
def reverse_score(score):
    values = score.split('–')
    return(str(values[1]) + '–' + str(values[0]))

def get_division(team, season):
    find = team_dict[(team_dict.Season == season) & (team_dict.Team == team)]
    try:
        #print(list(find['Division'])[0])
        return list(find['Division'])[0]
    except IndexError:
        return 'NA'
    
def score_split(string):
    if string[-1:] == ')':
        score = string.split(' ')[-1:]
        stripped_name = string.replace(score[0], '')[:-1]
        return(stripped_name, score[0])
    else:
        return(string, '0(0)')
    
def get_week(season, week):
    
    #pulls up page
    driver.get('https://quizcentral.net/qc/Online_Quiz_League_USA_-_Results_-_Season_' + str(season) + '_-_Week_' + str(week))
    time.sleep(5)
    
    #creates frame
    oql_frame = pd.DataFrame(columns=['Season', 'Week', 'Division', 'Date', 'Team', 'Opponent', 'Result', "Seat", 'Player'])

    error = 0
    i = 1

    while error == 0:
        try:
            table = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/div[3]/div[' + str(i) + ']')
            text = BeautifulSoup(table.get_attribute('outerHTML'), "html.parser")
            li_list = []
            teams = []
            team1 = []
            team2 = []

            for date in text.find_all("div", {"class": "date"}):
                matchdate = date.text[:10]

            for score in text.find_all("th", {"class": "score"}):
                match_score = score.text

            for team in text.find_all('a'):    
                teams.append(team.text)

            teams = teams[0:2]

            for j in text.find_all('li'):
                li_list.append(j.text)
                team1 = li_list[0:4]
                team2 = li_list[4:8]

            first = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div/div[3]/div[' + str(i) + ']/div[2]/ul/li[3]/a').text

            for k in range(0,4):
                newrow1 = {'Season': season, 'Week': week, 'Division': '', 'Date': matchdate, 'Team': teams[0], 'Opponent': teams[1], 'Result': match_score, 'Seat': k+1, 'Player': team1[k], 'First': first}
                newrow2 = {'Season': season, 'Week': week, 'Division': '', 'Date': matchdate, 'Team': teams[1], 'Opponent': teams[0], 'Result': reverse_score(match_score), 'Seat': k+1, 'Player': team2[k], 'First': first}
                oql_frame = oql_frame.append(newrow1, ignore_index = True)
                oql_frame = oql_frame.append(newrow2, ignore_index = True)
            
            i = i + 1
            
        except:
            error = 1
    
    if i > 1:
        name_list = []
        score_list = []

        for value in oql_frame.Player:
            if re.search('undefined', value):
                score = value.split(' ')[-1:][0]
                n = score[0]
                score = str(n) + '(0)'
                score_list.append(score)
                name_list.append(value.replace(str(n)+'(undefined)', ''))
            elif re.search('\d\(\d\)', value):
                score = value.split(' ')[-1:][0]
                score_list.append(score)
                name_list.append(value.replace(score, ''))
            else:
                name_list.append(value)
                score_list.append('0(0)')


        oql_frame['Player'] = name_list
        oql_frame['Captain'] = ["*" if '*' in i else '' for i in oql_frame.Player]
        oql_frame['Player'] = [str(i).replace('*', '') for i in oql_frame.Player]

        oql_frame['Score'] = score_list

        oql_frame['Points'] = [int(str(i).split('(')[0]) for i in oql_frame.Score]
        oql_frame['Direct'] = [int(str(i).split('(')[1].replace(')', '')) for i in oql_frame.Score]

        oql_frame['Side'] = ['a' if i == j else 'b' for i, j in zip(oql_frame.Team, oql_frame.First)]
        oql_frame['Seat'] = [str(i) + str(j) for i,j in zip(oql_frame.Seat, oql_frame.Side)]
        
        oql_frame = oql_frame.drop(['Side', 'First'], axis=1)
        return oql_frame
    
    else:
        return pd.DataFrame(columns=['Season', 'Week', 'Division', 'Date', 'Team', 'Opponent', 'Result',
       'Seat', 'Player', 'Captain', 'Score', 'Points', 'Direct'])

loops through the seasons

In [7]:
all_weeks = pd.DataFrame(columns=['Season', 'Week', 'Division', 'Date', 'Team', 'Opponent', 'Result',\
                                  'Seat', 'Player', 'Captain', 'Score', 'Points', 'Direct'])

for i in range(1,12):
    for j in range(1,22):
        frame = get_week(i,j)
        
        print(frame.head())
        
        all_weeks = pd.concat([all_weeks, frame])

  Season Week Division        Date                        Team  \
0      1    1           2020-10-01             4 Out of 5 Cats   
1      1    1           2020-10-01  Immune from Herd Mentality   
2      1    1           2020-10-01             4 Out of 5 Cats   
3      1    1           2020-10-01  Immune from Herd Mentality   
4      1    1           2020-10-01             4 Out of 5 Cats   

                     Opponent Result Seat            Player Captain  Score  \
0  Immune from Herd Mentality  59–35   1a    Colby Burnett        *  20(6)   
1             4 Out of 5 Cats  35–59   1b        Kaya Chua        *  10(4)   
2  Immune from Herd Mentality  59–35   2a    Dylan Minarik           14(5)   
3             4 Out of 5 Cats  35–59   2b  Charles McKeown            7(3)   
4  Immune from Herd Mentality  59–35   3a    Charlie Fritz           13(5)   

   Points  Direct  
0      20       6  
1      10       4  
2      14       5  
3       7       3  
4      13       5  
  Season Week 

KeyboardInterrupt: 

In [9]:
all_weeks.head()

,Season,Week,Division,Date,Team,Opponent,Result,Seat,Player,Captain,Score,Points,Direct
0,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,1a,Colby Burnett,*,20(6),20,6
1,1,1,,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,1b,Kaya Chua,*,10(4),10,4
2,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,2a,Dylan Minarik,,14(5),14,5
3,1,1,,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,2b,Charles McKeown,,7(3),7,3
4,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,3a,Charlie Fritz,,13(5),13,5


In [10]:
#just get one week

frame = get_week(3,20)
print(frame.head())
all_weeks = pd.concat([all_weeks, frame])

  Season Week Division        Date                        Team  \
0      3   20           2021-09-18             4 Out of 5 Cats   
1      3   20           2021-09-18  My Dinner With Andre' 3000   
2      3   20           2021-09-18             4 Out of 5 Cats   
3      3   20           2021-09-18  My Dinner With Andre' 3000   
4      3   20           2021-09-18             4 Out of 5 Cats   

                     Opponent Result Seat           Player Captain  Score  \
0  My Dinner With Andre' 3000  56–50   1b   Colby Burnett        *  10(4)   
1             4 Out of 5 Cats  50–56   1a  Ryan Rosenberg        *  12(5)   
2  My Dinner With Andre' 3000  56–50   2b   Dylan Minarik           18(5)   
3             4 Out of 5 Cats  50–56   2a     Will Alston            6(1)   
4  My Dinner With Andre' 3000  56–50   3b   Amanda Walker           12(4)   

   Points  Direct  
0      10       4  
1      12       5  
2      18       5  
3       6       1  
4      12       4  


In [11]:
all_weeks

,Season,Week,Division,Date,Team,Opponent,Result,Seat,Player,Captain,Score,Points,Direct
0,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,1a,Colby Burnett,*,20(6),20,6
1,1,1,,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,1b,Kaya Chua,*,10(4),10,4
2,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,2a,Dylan Minarik,,14(5),14,5
3,1,1,,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,2b,Charles McKeown,,7(3),7,3
4,1,1,,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,3a,Charlie Fritz,,13(5),13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,20,,2021-09-18,My Dinner With Andre' 3000,4 Out of 5 Cats,50–56,2a,Will Alston,,6(1),6,1
4,3,20,,2021-09-18,4 Out of 5 Cats,My Dinner With Andre' 3000,56–50,3b,Amanda Walker,,12(4),12,4
5,3,20,,2021-09-18,My Dinner With Andre' 3000,4 Out of 5 Cats,50–56,3a,Juliet Mayer,,11(4),11,4
6,3,20,,2021-09-18,4 Out of 5 Cats,My Dinner With Andre' 3000,56–50,4b,Charlie Fritz,,8(3),8,3


get teams by division

In [13]:
team_dict = pd.DataFrame(columns=['Season', 'Division', 'Team'])

for i in range(1,12):
    driver.get('https://quizcentral.net/qc/Online_Quiz_League_USA_-_Standings_-_Season_' + str(i))
    time.sleep(3)
    
    table = driver.find_element(By.ID, 'standingstable').get_attribute('outerHTML')

    team_div = pd.read_html(table)[0]

    team_div = team_div[['1', 'Team']]
    team_div.columns = ['Division', 'Team']

    team_div['Division'] = team_div['Division'].ffill().shift(1)
    team_div['Division'] = ['1' if i is np.NaN else i for i in team_div.Division]
    team_div['Season'] = i
    
    team_dict = pd.concat([team_dict, team_div])
    
team_dict.tail()

C:\Users\Zach Gozlan\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Division,Season,Team
115,5 Tiger,11,Stranger Danger
116,5 Tiger,11,The Island of Quizfit Toys
117,5 Tiger,11,Trust The Butt
118,5 Tiger,11,Standard Deviants
119,5 Tiger,11,Tiger Trivia Institute


Adds division and level to above table

In [14]:
all_weeks['Division'] = [get_division(i,j) for i,j in zip(all_weeks.Team, all_weeks.Season)]

In [15]:
all_weeks['Level'] = [(str(i)[0]) for i in all_weeks['Division']]

In [16]:
all_weeks

,Season,Week,Division,Date,Team,Opponent,Result,Seat,Player,Captain,Score,Points,Direct,Level
0,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,1a,Colby Burnett,*,20(6),20,6,1
1,1,1,1,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,1b,Kaya Chua,*,10(4),10,4,1
2,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,2a,Dylan Minarik,,14(5),14,5,1
3,1,1,1,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,2b,Charles McKeown,,7(3),7,3,1
4,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,3a,Charlie Fritz,,13(5),13,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,3,20,1,2021-09-18,My Dinner With Andre' 3000,4 Out of 5 Cats,50–56,2a,Will Alston,,6(1),6,1,1
4,3,20,1,2021-09-18,4 Out of 5 Cats,My Dinner With Andre' 3000,56–50,3b,Amanda Walker,,12(4),12,4,1
5,3,20,1,2021-09-18,My Dinner With Andre' 3000,4 Out of 5 Cats,50–56,3a,Juliet Mayer,,11(4),11,4,1
6,3,20,1,2021-09-18,4 Out of 5 Cats,My Dinner With Andre' 3000,56–50,4b,Charlie Fritz,,8(3),8,3,1


exports

In [16]:
all_weeks.to_csv('oql_s1s11_all_weeks.csv')

does sql over the frame to determine TOTW winners

In [17]:
first_level = pysqldf(''' SELECT all_weeks.* from all_weeks
            INNER JOIN
            (SELECT Season, Week, Seat, Level, max(Points) as 'Points'
            FROM all_weeks 
            group by Season, Week, Seat, Level) max_points
            on max_points.Season = all_weeks.Season
            AND max_points.Week = all_weeks.Week
            AND max_points.Seat = all_weeks.Seat
            AND max_points.Points = all_weeks.Points
            AND max_points.Level = all_weeks.Level
            order by Season, Week, Seat, Level
            ''')

pysqldf(''' SELECT first_level.* from first_level
            INNER JOIN
            (SELECT Season, Week, Seat, Level, max(Direct) as 'Direct'
            FROM first_level
            group by Season, Week, Seat, Level) max_directs
            on max_directs.Season = first_level.Season
            AND max_directs.Week = first_level.Week
            AND max_directs.Seat = first_level.Seat
            AND max_directs.Direct = first_level.Direct
            AND max_directs.Level = first_level.Level
            order by Season, Week, Seat, Level
            ''').to_csv('totw_test.csv')

In [18]:
first_level = pysqldf(''' SELECT all_weeks.* from all_weeks
            INNER JOIN
            (SELECT Season, Week, Seat, Level, max(Direct) as 'Direct'
            FROM all_weeks 
            group by Season, Week, Seat, Level) max1
            on max1.Season = all_weeks.Season
            AND max1.Week = all_weeks.Week
            AND max1.Seat = all_weeks.Seat
            AND max1.Direct = all_weeks.Direct
            AND max1.Level = all_weeks.Level
            order by Season, Week, Seat, Level
            ''')

second_level = pysqldf(''' SELECT first_level.* from first_level
            INNER JOIN
            (SELECT Season, Week, Seat, Level, max(Points) as 'Points'
            FROM first_level
            group by Season, Week, Seat, Level) max2
            on max2.Season = first_level.Season
            AND max2.Week = first_level.Week
            AND max2.Seat = first_level.Seat
            AND max2.Points = first_level.Points
            AND max2.Level = first_level.Level
            order by Season, Week, Level, Seat
            ''')

second_level.to_csv('totw_test.csv')

In [20]:
all_weeks = pd.read_csv('oql_s1s11_all_weeks.csv')

all_weeks

,Unnamed: 0,Season,Week,Division,Date,Team,Opponent,Result,Seat,Player,Captain,Score,Points,Direct,Level
0,0,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,1a,Colby Burnett,*,20(6),20,6,1
1,1,1,1,1,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,1b,Kaya Chua,*,10(4),10,4,1
2,2,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,2a,Dylan Minarik,NaN,14(5),14,5,1
3,3,1,1,1,2020-10-01,Immune from Herd Mentality,4 Out of 5 Cats,35–59,2b,Charles McKeown,NaN,7(3),7,3,1
4,4,1,1,1,2020-10-01,4 Out of 5 Cats,Immune from Herd Mentality,59–35,3a,Charlie Fritz,NaN,13(5),13,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50635,443,11,10,5 Tiger,2024-03-17,WAP And Peace,Stranger Danger,49–38,2a,Kaya Blauvelt,NaN,19(7),19,7,5
50636,444,11,10,5 Tiger,2024-03-17,Stranger Danger,WAP And Peace,38–49,3b,Nathaniel Tan,*,9(3),9,3,5
50637,445,11,10,5 Tiger,2024-03-17,WAP And Peace,Stranger Danger,49–38,3a,Dave Cutler,NaN,5(2),5,2,5
50638,446,11,10,5 Tiger,2024-03-17,Stranger Danger,WAP And Peace,38–49,4b,Luc Leavenworth,NaN,10(4),10,4,5
